In [57]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import os

def import_from_excel(e_file, filename):
    kinase_targets = e_file['kinase target'].tolist()
    if kinase_targets != []:
        kinase_targets = kinase_targets[0]#just the top score
    JS_scores = e_file['JS Distance Score'].tolist() #just the top score
    if JS_scores != []:
        JS_scores = JS_scores[0]
    #off_targets = e_file['off-target %inhibition values'].tolist()
    mu = int(filename.split('.')[0].split('_')[-2])
    return [mu, kinase_targets, JS_scores] #off_targets]

test_results_200 = []
test_results_700 = []

test_results = {}

for filename in os.listdir(os.getcwd()):
    if filename[0:8] == '030822_i':
        f = './'+str(filename)
        replicate_results = []
        targets = ''
        for x in range(1,4):
            open_file = pd.read_excel(f, str(x), engine='openpyxl')
            inhibitor_combination = import_from_excel(open_file, filename)
            targets = inhibitor_combination[1]
            replicate_results.append((x,inhibitor_combination))
        if targets != []:
            if targets not in test_results:
                test_results[targets] = [[],[]]
            if replicate_results[0][1][0] == 200:
                test_results[targets][0].append(replicate_results)
                #test_results_200.append(replicate_results)
            elif replicate_results[0][1][0] == 700:
                test_results[targets][1].append(replicate_results)
                #test_results_700.append(replicate_results)
        
#check to make sure we have 5 sets of results for each  
for k in test_results:
    print(len(test_results[k][0]))
    print(len(test_results[k][1]))

0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3


3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0


In [58]:
#sig results

sig_results = {}

for k in test_results:
    res_200 = test_results[k][0]
    res_700 = test_results[k][1]
    
    #get the best i=1 or i=2 score to compare to... either the most selective single inhibitor
    #or the most selective single inhibitor for each kinase
    
    #get the average i=1 score (most selective )
    i_1_200_vals = [r[0][1][2] for r in res_200 if r[0][0] == 1]
    i_1_700_vals = [r[0][1][2] for r in res_700 if r[0][0] == 1]
    
    i_2_200_vals = [r[1][1][2] for r in res_200 if r[1][0] == 2]
    i_2_700_vals = [r[1][1][2] for r in res_700 if r[1][0] == 2]
        
    sig_200_scores = []
    sig_700_scores = []
    
    sig_dist_200 = 0
    sig_dist_700 = 0
    
    if res_200 != []:
    
        #get the vals at each higher combo number. test if sig. if so, save those vals
        for i in range(1, len(res_200[0])):
            
            i_vals = [r[i][1][2] for r in res_200]
            
            #compare to i=1 or i=2, must be significantly better than both
            test_set = []
            if np.mean(i_1_200_vals) > np.mean(i_2_200_vals):
                test_set = i_1_200_vals
            else:
                test_set = i_2_200_vals
                
            #do we have a test set? if not, automatically significant
            if test_set != []:
            
                #test if sig diff or not
                tval, pval = stats.ttest_ind(test_set, i_vals)
            
                #if sig and absolute diff is > 0.001
                #if (pval < 0.05) and (np.mean(test_set)+0.001 < np.mean(i_vals)):
                if np.mean(test_set)+0.001 < np.mean(i_vals):
                    
                    #if this is better than our previous best score...
                    if np.mean(i_vals) - np.mean(test_set) > sig_dist_200:
                        sig_200_scores = i_vals
                        sig_dist_200 = (np.mean(i_vals) - np.mean(test_set))   
            else:
                #so we don't have a test set, will be significant, check to see if it is beter than our last
                    if np.mean(i_vals) > sig_dist_200:
                        sig_200_scores = i_vals
                        sig_dist_200 = np.mean(i_vals)
                        
    if res_700 != []:
    
        #get the vals at each higher combo number. test if sig. if so, save those vals
        for i in range(1, len(res_700[0])):
            
            i_vals = [r[i][1][2] for r in res_700]
            
            #compare to i=1 or i=2, must be significantly better than both
            test_set = []
            if np.mean(i_1_700_vals) > np.mean(i_2_700_vals):
                test_set = i_1_700_vals
            else:
                test_set = i_2_700_vals
                
            #do we have a test set? if not, automatically significant
            if test_set != []:
            
                #test if sig diff or not
                tval, pval = stats.ttest_ind(test_set, i_vals)
            
                #if sig and absolute diff is > 0.001
                if (pval < 0.05) and (np.mean(test_set)+0.001 < np.mean(i_vals)):
                    
                    #if this is better than our previous best score...
                    if np.mean(i_vals) - np.mean(test_set) > sig_dist_700:
                        sig_700_scores = i_vals
                        sig_dist_700 = np.mean(i_vals) - np.mean(test_set)  
            else:
                #so we don't have a test set, will be significant, check to see if it is beter than our last
                    if np.mean(i_vals) > sig_dist_700:
                        sig_700_scores = i_vals
                        sig_dist_700 = np.mean(i_vals)  
                        
    if sig_200_scores != [] or sig_700_scores != []:
        for iter in test_results[k][1][0]:
            if iter[1][1] != []:
                knames = iter[1][1]
        knames = knames.strip('][').replace('\'', '').replace(', ', '_')
        sig_results[knames] = [sig_200_scores,sig_700_scores,sig_dist_200,sig_dist_700]
        
for key in sig_results:
    print(key, sig_results[key])

ADCK1_TANK [[], [0.7410089059820573, 0.737479403465644, 0.7393763932900169], 0, 0.005902281463063042]
CAMK2G_TBKBP1 [[], [0.8132062747775922, 0.81355567868462, 0.8131442547439489], 0, 0.0049119157678734204]
CAMKK2_STK4 [[], [0.7525772338981702, 0.7519014164876227, 0.7517481577129174], 0, 0.0037703110058746603]
CDK4_PRKD2 [[], [0.7866878949387092, 0.7898613222303023, 0.7892946501150918], 0, 0.007265930652939567]
CIT_TESK1 [[], [0.7574980310411082, 0.7577788079973622, 0.7573274377741815], 0, 0.0016651001054657222]
MAP4K4_PRKD2 [[], [0.7857991236005026, 0.7862845648071803, 0.7870059258027587], 0, 0.005270002368977678]
MAP4K4_PRKD3 [[], [0.7790702536117527, 0.7771062909558556, 0.7791719611609577], 0, 0.007442785789123607]
MAP4K4_MAP3K11 [[], [0.7761548904577269, 0.776474472219688, 0.7792975093496844], 0, 0.003859656989644744]
MAP4K4_ULK1 [[], [0.749021154702341, 0.7459942473621658, 0.7489699635805039], 0, 0.007825829028472175]
MAP4K4_STK3 [[], [0.7706435302737948, 0.77104669374436, 0.76928